In [1]:
!pip install pygame

  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/66/57/1311ff5bbd64093795f64c66910bbc12b7c5d83ca95766cce7ba501ff7e7/pygame-2.5.2-cp312-cp312-win_amd64.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 325.1 kB/s eta 0:00:34
   ---------------------------------------- 0.0/10.8 MB 279.3 kB/s eta 0:00:39
   ---------------------------------------- 0.1/10.8 MB 456.6 kB/s eta 0:00:24
   ---------------------------------------- 0.1/10.8 MB 504.4 kB/s eta 0:00:22
   ---------------------------------------- 0.1/10.8 MB 462.0 kB/s eta 0:00:24
    --------------------------------------- 0.2/10.8 MB 523.5 kB/s eta 0:00:21
    --------------------------------------- 0.2/10.8 MB 562.0 kB/s eta 0:00:19
    --------------------------------------- 0.3/10.8 MB 603.8 kB/s eta 0:00:18
   - -------------------------------------- 0.3/10.8 MB 723.4 kB/s eta 0:00:15
   - -------------------------------------- 0.5/10.8 MB 893.0 kB/s eta 0:00:12
   -- ------------------------------------- 0.7/10.8 MB 1.1 MB/s eta 0

In [5]:
import pygame 
import sys

BLACK = (0, 0, 0)       # For displaying text on screen
GREY = (180, 180, 140)  # For empty disk
RED = (255, 0, 0)       # For Human move
BLUE = (0, 80, 239)      # For Background grid
YELLOW = (255, 255, 0)  # For AI move
ROWS = 6
COLS = 7
WIN = 4    
WINDOW_WIDTH = 605
WINDOW_HEIGHT = 520
MARGIN = 10

DISC_SIZE = 80
DISC_GAP = 5

BOARD_WIDTH = COLS * (DISC_SIZE + DISC_GAP) + MARGIN
BOARD_HEIGHT = ROWS * (DISC_SIZE + DISC_GAP) + MARGIN

HUMAN = 1
AI = -1
EMPTY = 0

# alpha and beta values
INFINITY = float('inf')
ALPHA = -INFINITY
BETA = INFINITY

# Creating window
pygame.init()
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption('Connect Four')
clock = pygame.time.Clock()

font = pygame.font.SysFont('Arial', 60)

# initializing game board with zeros
board = []
for i in range(ROWS):
    row = []
    for j in range(COLS):
        row.append(EMPTY)
    board.append(row)

current_player = HUMAN

# stores the game state (running or over)
game_state = 'running'

# stores the game result (win, lose, draw or None)
game_result = None

def draw_board():
    # Fills the window with grey color
    window.fill(GREY)

    # draw a blue rectangle for the background
    pygame.draw.rect(window, BLUE, (0, 0, BOARD_WIDTH, BOARD_HEIGHT))

    # draw colored discs on the board
    for row in range(ROWS):
        for col in range(COLS):
            # calculate the position and radius of each disc slot
            x = MARGIN + (col * (DISC_SIZE + DISC_GAP)) + (DISC_SIZE // 2)
            y = MARGIN + (row * (DISC_SIZE + DISC_GAP)) + (DISC_SIZE // 2)
            r = DISC_SIZE // 2 

            if board[row][col] == HUMAN:
                # draw a red circle for the human disc
                pygame.draw.circle(window, RED, (x, y), r)
            elif board[row][col] == AI:
                # draw a yellow circle for the AI disc
                pygame.draw.circle(window, YELLOW, (x, y), r)
            else:
                # draw a grey circle for each empty disc slot
                pygame.draw.circle(window, GREY, (x, y), r)

    # check the game state
    if game_state == 'over':
        if game_result == 'win':
            text = font.render('YOU WIN!', True, BLACK)
        elif game_result == 'lose':
            text = font.render('YOU LOSE!', True, BLACK)
        elif game_result == 'draw':
            text = font.render('DRAW!', True, BLACK)

        # draw the text rectangle and place it in the centre
        text_rect = text.get_rect()
        text_rect.center = (WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2)
        window.blit(text, text_rect)


def is_full():
    for col in range(COLS):
        if board[0][col] == EMPTY:
            return False
    return True

def is_winner(player):
    for row in range(ROWS):
        for col in range(COLS):
            if board[row][col] == player:
                # check horizontal line for player
                if col + WIN <= COLS:
                    h_count = 0   #horizontal disc count
                    for i in range(WIN):
                        # check if the next slot belongs to the player 
                        if board[row][col + i] == player:
                            h_count += 1

                    if h_count == WIN:
                        return True

                # check vertical line for player
                if row + WIN <= ROWS:
                    v_count = 0    #vertical disk count
                    for i in range(WIN):
                        # Check if the next slot belongs to the player
                        if board[row + i][col] == player:
                            v_count += 1

                    if v_count == WIN:
                        return True

                 # check diagonal line for player (negative slope)
                if row + WIN <= ROWS and col + WIN <= COLS:
                     d_count = 0    #diagonal disk count
                     for i in range(WIN):
                         # check if the slot belongs to the player
                         if board[row + i][col + i] == player:
                             d_count += 1

                     if d_count == WIN:
                         return True

                 # check diagonal line for player (positive slope)
                if row - WIN >= -1 and col + WIN <= COLS:
                     d_count = 0     #diagonal disk count
                     for i in range(WIN):
                         # check if the slot belongs to the player
                         if board[row - i][col + i] == player:
                             d_count += 1

                     if d_count == WIN:
                         return True

    # return false if no winner
    return False

# Function to generate all possible moves for a given board state and player
def generate_moves(board, player):
    moves = []
    for col in range(COLS):
        if board[0][col] == EMPTY:
            for row in range(ROWS - 1, -1, -1):
                if board[row][col] == EMPTY:
                    # create a copy of the board
                    new_board = [row[:] for row in board]
                    # drop the player's disc into the empty slot
                    new_board[row][col] = player
                    moves.append((col, new_board))
                    break
                    
    return moves

# evaluate scoring heuristic
def evaluate(board):
    score = 0
    for row in range(ROWS):
        for col in range(COLS):
            if board[row][col] != EMPTY:
                # check horizontal line
                if col + WIN <= COLS:
                    red_count = 0
                    yellow_count = 0
                    for i in range(WIN):
                        if board[row][col + i] == HUMAN:
                            red_count += 1
                        elif board[row][col + i] == AI:
                            yellow_count += 1

                    # calculate the line value based on the number of discs in the line (more discs = larger value)
                    if red_count == 0 and yellow_count > 0:
                        line_value = 10 ** (yellow_count - 1) 
                    elif red_count > 0 and yellow_count == 0:
                        line_value = -10 ** (red_count - 1) 
                    elif red_count > yellow_count:
                        line_value = -1
                    elif red_count < yellow_count:
                        line_value = 1
                    else:
                        line_value = 0

                    score += line_value

                # check vertical line
                if row + WIN <= ROWS:
                    red_count = 0
                    yellow_count = 0
                    for i in range(WIN):
                        if board[row + i][col] == HUMAN:
                            red_count += 1
                        elif board[row + i][col] == AI:
                            yellow_count += 1

                    # calculate the line value based on the number of discs in the line (more discs = larger value)
                    if red_count == 0 and yellow_count > 0:
                        line_value = 10 ** (yellow_count - 1)
                    elif red_count > 0 and yellow_count == 0:
                        line_value = -10 ** (red_count - 1)
                    elif red_count > yellow_count:
                        line_value = -1
                    elif red_count < yellow_count:
                        line_value = 1
                    else:
                        line_value = 0

                    score += line_value

                 # check diagonal line (positive slope)
                if row + WIN <= ROWS and col + WIN <= COLS:
                     red_count = 0
                     yellow_count = 0
                     for i in range(WIN):
                         if board[row + i][col + i] == HUMAN:
                             red_count += 1
                         elif board[row + i][col + i] == AI:
                             yellow_count += 1

                     # calculate the line value based on the number of discs in the line (more discs = larger value)
                     if red_count == 0 and yellow_count > 0:
                         line_value = 10 ** (yellow_count - 1) 
                     elif red_count > 0 and yellow_count == 0:
                         line_value = -10 ** (red_count - 1)
                     elif red_count > yellow_count:
                         line_value = -1
                     elif red_count < yellow_count:
                         line_value = 1
                     else:
                         line_value = 0

                     score += line_value

                 # check diagonal line (negative slope)
                if row - WIN >= -1 and col + WIN <= COLS:
                     red_count = 0
                     yellow_count = 0
                     for i in range(WIN):
                         if board[row - i][col + i] == HUMAN:
                             red_count += 1
                         elif board[row - i][col + i] == AI:
                             yellow_count += 1

                     # calculate the line value based on the number of discs in the line (more discs = larger value)
                     if red_count == 0 and yellow_count > 0:
                         line_value = 10 ** (yellow_count - 1) 
                     elif red_count > 0 and yellow_count == 0:
                         line_value = -10 ** (red_count - 1) 
                     elif red_count > yellow_count:
                         line_value = -1
                     elif red_count < yellow_count:
                         line_value = 1
                     else:
                         line_value = 0

                     score += line_value
                    
    return score

def alpha_beta(board, depth, alpha, beta, player):
    # Check if the game is over or the depth limit is reached
    if is_winner(HUMAN) or is_winner(AI) or is_full() or depth == 0:
        return (evaluate(board), None)

    # AI (maximizing player)
    if player == AI:
        best_score = -INFINITY
        best_move = None
        moves = generate_moves(board, AI)
        for move in moves:
            col, new_board = move
            # recursively call the alpha beta function with the new board state, decreased depth, and switched player
            score, _ = alpha_beta(new_board, depth - 1, alpha, beta, HUMAN)
            if score > best_score:
                best_score = score
                best_move = col

            alpha = max(alpha, best_score)
            # Check if beta is less than alpha (pruning condition)
            if beta < alpha:
                break

        return (best_score, best_move)

    # human (minimizing player)
    elif player == HUMAN:
        best_score = INFINITY
        best_move = None
        moves = generate_moves(board, HUMAN)
        for move in moves:
            col, new_board = move
            # recursively call the alpha beta function with the new board state, decreased depth, and switched player
            score, _ = alpha_beta(new_board, depth - 1, alpha, beta, AI)
            if score < best_score:
                best_score = score
                best_move = col

            beta = min(beta, best_score)
            # Check if beta is less than alpha (pruning condition)
            if beta < alpha:
                break

        return (best_score, best_move)

# create a main loop 
running = True

try: 
 while running:

    # Process events (keystrokes, mouse clicks, etc.)
    for event in pygame.event.get():
        # check if the event is the quit event
        if event.type == pygame.QUIT:
            # exit the game
            sys.exit()

        # check if the event is a mouse click
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if game_state == 'running' and current_player == HUMAN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                # check if the mouse position is within the board area
                if mouse_x <= BOARD_WIDTH and  mouse_y <= BOARD_HEIGHT:
                    # calculate the column index based on the mouse position
                    col = (mouse_x) // (DISC_SIZE + DISC_GAP)
                    # check if the top row of the column is empty
                    if board[0][col] == EMPTY:
                        for row in range(ROWS - 1, -1, -1):
                            if board[row][col] == EMPTY:
                                # drop the human disc into the empty slot
                                board[row][col] = HUMAN
                                # switch the current player to AI
                                current_player = AI
                                break

    # check for winners or full board
    if game_state == 'running':
        if is_winner(HUMAN):
            game_state = 'over'
            game_result = 'win'
        elif is_winner(AI):
            game_state = 'over'
            game_result = 'lose'
        elif is_full():
            game_state = 'over'
            game_result = 'draw'
        else:
            if current_player == AI:
                # set a difficulty level as the depth limit for alpha beta pruning (higher = harder)
                difficulty = 6
                _, best_move = alpha_beta(board, difficulty, ALPHA, BETA, AI)
                if best_move is not None:
                    for row in range(ROWS - 1, -1, -1):
                        if board[row][best_move] == EMPTY:
                            board[row][best_move] = AI
                            current_player = HUMAN
                            break


    draw_board()

    # update the window
    pygame.display.flip()

    # set the frame rate
    clock.tick(60)

except SystemExit:
    pygame.quit()